In [81]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [82]:
# Load data
data = pd.read_csv("IRIS.csv")

# Encode the output column
encoder = LabelEncoder()
encoder.fit(data["species"])
data["species"] = encoder.transform(data["species"])

print(data)

     sepal_length  sepal_width  petal_length  petal_width  species
0             5.1          3.5           1.4          0.2        0
1             4.9          3.0           1.4          0.2        0
2             4.7          3.2           1.3          0.2        0
3             4.6          3.1           1.5          0.2        0
4             5.0          3.6           1.4          0.2        0
..            ...          ...           ...          ...      ...
145           6.7          3.0           5.2          2.3        2
146           6.3          2.5           5.0          1.9        2
147           6.5          3.0           5.2          2.0        2
148           6.2          3.4           5.4          2.3        2
149           5.9          3.0           5.1          1.8        2

[150 rows x 5 columns]


In [83]:
# Split into input and target
X = data.drop("species", axis = 1).values #.values is needed to convert to np array
Y = data["species"].values

# Convert to tensors
X = torch.tensor(X, dtype = torch.float32)
Y = torch.tensor(Y, dtype = torch.long)

# Split into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [84]:
# Define neural network

class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.relu = nn.ReLU()

        self.layer1 = nn.Linear(4, 32)
        
        self.layer2 = nn.Linear(32, 3)

        self.dropout = nn.Dropout(p = 0.2)

    def forward(self, x):
        return self.layer2(self.dropout(self.relu(self.layer1(x))))

model = IrisModel()

In [85]:
# Loss and optimizer
learning_rate = 0.1
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

# Training loop
epochs = 100

for epoch in range(epochs):
    optimizer.zero_grad() # Reset stored gradients

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)

    # Backwards pass
    loss.backward()
    optimizer.step()


In [86]:
for i in range(5):
    output = model(X_test[i])
    _, predicted = torch.max(output, axis = 0)
    expected = Y_test[i]
    print(f"Expected: {expected} | Predicted: {predicted.item()}")

Expected: 2 | Predicted: 2
Expected: 1 | Predicted: 1
Expected: 0 | Predicted: 0
Expected: 2 | Predicted: 2
Expected: 0 | Predicted: 0


In [87]:
# Evaluate

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == Y_test).sum().item() / len(Y_test)
    print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 96.67%
